# Vorbereitung des Datensatzes

In [1]:
import pandas as pd
import numpy as np

In der Folge werden keine Ländergruppen und nur die Jahre 1990 bis 2020 betrachet. Von dem verbliebenen Datensatz werden nur jene Indikatoren behalten, die mindesten 20% gefüllt sind.

In [2]:
def reset_base():
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    idx = pd.IndexSlice
    
    #loading original data
    base= pd.read_csv('../Data/WDIData.csv') #see downloads worldbank
    base = base.drop(['Country Code', 'Indicator Code', 'Unnamed: 66'], axis=1) #name of column 'Unnamed: 66' may differ
        
    #dropping regions
    countries = pd.read_csv('additional_data/countries.csv').drop('Unnamed: 0', axis=1)
    base = pd.merge(base, countries, how='left')
    base = base.loc[base['Type'] != 'Region'].drop('Type', axis=1)
    
    #keeping only years 1990-2020
    base = base.set_index(['Country Name', 'Indicator Name'])
    base = base.loc[:, '1990':'2020']

    #dropping countries and territories with less than 1 million occupants
    x = base.index.get_level_values('Country Name').unique()[base.loc[idx[:, 'Population, total'], '2014'].values > 1000000]
    base = base[base.index.get_level_values('Country Name').isin(x) == True]
    
    #dropping all indicators with more then 50% missing values
    keep = pd.DataFrame(pd.DataFrame(base.isna().groupby('Indicator Name').sum()).T.sum(), columns=['NaN'])
    keep = keep.loc[keep['NaN'] <len(base.index.get_level_values('Country Name').unique())*len(base.columns)*0.5] #kept if 80% of entries are not NaN
    base = base.loc[idx[:, keep.index], :]
    return base



In [3]:
base = reset_base()

In [4]:
print(f'Number of countries remaining: {len(base.index.get_level_values("Country Name").unique())}')
print(f'Number of indicators remaining: {len(base.index.get_level_values("Indicator Name").unique())}')

Number of countries remaining: 159
Number of indicators remaining: 850


In [5]:
print(f"Number of NaNs still in Dataset: {base.isna().sum().sum()}")
print(f"Percentage of missing values: {base.isna().sum().sum()/base.notna().sum().sum()}")

Number of NaNs still in Dataset: 870053
Percentage of missing values: 0.26209597128808104


Die verbliebenen Indikatoren werden hier aufgeführt. Dazu ist zu sehen, wie viele Einträge für diesen Indikator in den Jahren 1990 bis 2020 fehlen und in welche thematischen Bereiche die Indikatoren fallen. Aus dieser Auslistung wurden nun manuel Indikatoren ausgewählt, die 
- verhältnismäßig gut ausgefüllt sind,
- alle Themenbereiche abdecken und 
- Relevanz für die kommenden Analysen haben.

In [6]:
def get_indicator_nans(df):
    df = df.apply(lambda x: x.replace('"', ''))
    df = df.reset_index()
    size = df.loc[df['Indicator Name']==df['Indicator Name'].unique()[1]].size
    
    
    topics = pd.read_csv('additional_data/topics_done.csv')
        
    nan = [df.loc[df['Indicator Name']==i].isna().sum().sum() for i in df['Indicator Name'].unique()]
    n = [df.loc[df['Indicator Name']==i].notna().sum().sum() for i in df['Indicator Name'].unique()]
    
    df = pd.DataFrame({'indicators':df['Indicator Name'].unique(), 'nan':nan, 'n':n})
    new = pd.merge(df.sort_values('nan'), topics, how='left', left_on='indicators', right_on='Indicator Name')
    new.drop(['Unnamed: 0', 'Indicator Name','Indicator Code'], inplace=True, axis=1)
    new.set_index(['Topic description', 'indicators']).to_csv('additional_data/low_nan_high_nan.csv', sep=';', decimal=',')
    return new

get_indicator_nans(base)

,indicators,nan,n,Topic description
0,"Population ages 15-64, total",0,5247,Social: population
1,Sex ratio at birth (male births per female bir...,0,5247,Social: population
2,"Population ages 20-24, female (% of female pop...",0,5247,Social: population
3,Population ages 0-14 (% of total population),0,5247,Social: population
4,"Population ages 0-14, female",0,5247,Social: population
...,...,...,...,...
845,"Secondary education, teachers",2418,2829,Social: education
846,"Net bilateral aid flows from DAC donors, Irela...",2423,2824,Debt: aid flows from DAC
847,"PPG, IDA (DOD, current US$)",2450,2797,Debt: external
848,Grants and other revenue (current LCU),2450,2797,Government finance: IMF


Die manuell ausgewählten Indikatoren werden hier eingelesen.

In [7]:
new = pd.read_csv('additional_data/chosen_indicators.csv', sep=';')
new.dropna(subset=['keep'], inplace=True)
new.drop('keep', axis=1, inplace=True)

In [8]:
base = base.loc[base.index.isin(new['indicators'], level='Indicator Name')]

In [9]:
print(f"Percentage of missing values: {base.isna().sum().sum()/base.notna().sum().sum()}")

Percentage of missing values: 0.24164230034555872


In [10]:
print(f'Number of countries remaining: {len(base.index.get_level_values("Country Name").unique())}')
print(f'Number of indicators remaining: {len(base.index.get_level_values("Indicator Name").unique())}')

Number of countries remaining: 159
Number of indicators remaining: 163


In [11]:
base.head(5)

,,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Country Name,Indicator Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.8000,23.0000,24.8000,26.1000,27.4000,28.6000,29.7000,30.9000,31.9000,33.2000
Albania,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.9000,71.3000,73.1500,74.6000,76.2000,77.8000,78.7000,79.8000,80.7000,81.3000
Algeria,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.3000,99.4000,99.4000,99.5000,99.5000,99.6000,99.6000,99.6000,99.7000,99.7000
Angola,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,44.8000,45.2000,45.8000,46.5000,47.2000,47.8000,48.4000,49.1000,49.5000,50.2000
Argentina,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.3000,99.5000,99.6000,99.7000,99.7000,99.8000,99.8000,99.8000,99.9000,99.9000


Um nicht in allen künftigen Analysen den Gesammtdatensatz einlesen und wie hier transformieren zu müssen, wird der transformierte Datensatz gespeichert.

In [12]:
#base.to_csv('additional_data/base.csv')